In [16]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

Se importan las galerias necesarias, y se crea un dataframe para visualizar los metadatos de Yelp.

In [17]:
# Ruta del archivo .pkl
file_path = r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\data parcial\business.pkl'

# Cargar el archivo .pkl
df_business_yelp_base = pd.read_pickle(file_path)
df_business_yelp_base.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_business_yelp_base.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'business_id', 'name', 'address',
       'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

Filtramos por el Estado de California.

In [24]:
df_california = df_business_yelp_base[df_business_yelp_base['state'] == 'CA']
df_california.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
29,sqSqqLy0sN8n2IZrAbzidQ,Domino's Pizza,3001 Highway 31 W,White House,CA,37188,36.464747,-86.659187,3.5,8,1,{'RestaurantsPriceRange2': '1'},"Pizza, Chicken Wings, Sandwiches, Restaurants","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ..."
88,LhpPSrulqVeTyJeK2xydvQ,Fresh Fruits & Salads,114 N 3rd St,Camden,CA,08102,39.94669,-75.123327,4.5,6,1,"{'BusinessParking': '{'garage': False, 'street...","Juice Bars & Smoothies, Restaurants, Fruits & ...","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
94,ZM46RDLXaFNo_z6t-j_L4w,Absolutely Perfect Inc,1153 Byberry Rd,Bensalem,CA,19020,40.113121,-74.914597,3.5,5,1,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...","Masonry/Concrete, Gardeners, Lawn Services, Tr...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ..."
123,iPSPcprSnkLOuL1Ns8UZZA,Pete's Barber Shop,1209 N 29th St,Philadelphia,CA,19121,39.97488,-75.183262,4.5,12,1,"{'RestaurantsPriceRange2': '1', 'BusinessParki...","Barbers, Beauty & Spas",None
144,PYW8Buz8S8LpjIe6mZTAcw,Payless Liquors,954 N State Rd 135,Greenwood,CA,46143,39.63228,-86.158674,3.5,13,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Food, Beer, Wine & Spirits",None
173,E-nhxuu3zbt02oCj_1AFng,Selah,"510 N 7th Ave, Ste 130",Tucson,CA,85705,32.228092,-110.969804,5.0,60,1,"{'WheelchairAccessible': 'True', 'BikeParking'...","Waxing, Hair Removal, Skin Care, Eyelash Servi...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
222,8c0r7olQSYGcws0bTd3ikw,Zesty Tsunami,6614 Osteen Rd,New Port Richey,CA,34653,28.250218,-82.682441,4.0,5,0,"{'RestaurantsReservations': 'False', 'Restaura...","Hawaiian, Food, Food Trucks, Street Vendors, R...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
230,do6EpBc7ZUoaqdzuUuGFXA,Jim's Auto Detailing,729 W Lancaster Ave,Wayne,CA,19087,40.04752,-75.409145,4.5,8,1,None,"Automotive, Auto Detailing, Car Wash, Auto Gla...",None
404,pYC5fKcpj8GKjSAc7Sl3RQ,Main Exchange Barber Shop,3108 N Boundary Blvd,Tampa,CA,33602,27.862363,-82.498121,2.0,7,1,"{'RestaurantsPriceRange2': '1', 'BikeParking':...","Barbers, Beauty & Spas",None


Si bien existen más de 5000 registros, vemos que las ciudades no coinciden con el Estado de California. Son de múltiples Estados, por lo que hay un error en la columna 'state'.

In [25]:
df_business_yelp_base['state'].describe()

count     150343
unique        27
top           PA
freq       34039
Name: state, dtype: object

In [26]:
df_business_yelp_base['city'].describe()

count           150346
unique            1416
top       Philadelphia
freq             14569
Name: city, dtype: object

Se prueba hacer un filtrado por las diferentes ciudades del Estado de California, para identificar aquellos registros que efectivamente pertenezcan a ese Estado.

In [27]:
ciudades = ['Los Angeles', 'San Francisco', 'San Diego', 'San Jose', 'Sacramento', 'Fresno', 'Long Beach', 'Oakland',
    'Bakersfield', 'Anaheim', 'Santa Ana', 'Riverside', 'Stockton', 'Chula Vista', 'Irvine', 'Fremont', 
    'San Bernardino', 'Modesto', 'Fontana', 'Moreno Valley', 'Huntington Beach', 'Glendale', 'Oxnard', 
    'Ontario', 'Rancho Cucamonga', 'Santa Clarita', 'Oceanside', 'Garden Grove', 'Elk Grove', 'Corona',
    'Lancaster', 'Palmdale', 'Hayward', 'Salinas', 'Pomona', 'Sunnyvale', 'Escondido', 'Torrance', 
    'Pasadena', 'Orange', 'Fullerton', 'Thousand Oaks', 'Visalia', 'Simi Valley', 'Concord', 
    'Roseville', 'Victorville', 'Vallejo', 'Berkeley', 'Fairfield'
]
# Eliminar las columnas duplicadas
df_business_yelp_base = df_business_yelp_base.loc[:, ~df_business_yelp_base.columns.duplicated()]
# Filtrar las filas donde la columna 'city' esté en la lista de ciudades
df_ciudades = df_business_yelp_base[df_business_yelp_base['city'].isin(ciudades)]
df_ciudades.head()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
762,mYbEGFsRcA0tfUNSYcBJMA,Golden River Restaurant,"8 E Scott St, Ste 10",Riverside,FL,08075,40.037703,-74.959281,5.0,8,1,"{'RestaurantsReservations': 'False', 'Restaura...","American (New), Restaurants",None
2072,xjF8NZZ2WIk3A2sW3nXyhA,Lambert International Airport Cell Phone Lot T...,9896 Air Cargo Rd,Berkeley,TN,63134,38.734285,-90.348893,4.0,13,1,"{'BusinessAcceptsCreditCards': 'False', 'WiFi'...","Automotive, Parking, Airports, Hotels & Travel",None
22467,dWix751x3N2wKv7Qwn1zDA,Genovesi Glass & Mirror,560 S Fairview St,Riverside,LA,08075,40.031,-74.9561,5.0,6,1,{'BusinessAcceptsCreditCards': 'True'},"Windows Installation, Home Services, Contracto...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
22793,Zk1lvwSdMk8HeXMAPQBUqw,Delta Movers,,Berkeley,AZ,63130,38.750336,-90.342778,1.5,28,1,"{'BusinessAcceptsCreditCards': 'False', 'ByApp...","Self Storage, Home Services, Local Services, M...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
28393,6iHvWggONIDt8cKXJGz0VQ,The Madison,33 Lafayette St,Riverside,CA,08075,40.038566,-74.960761,3.5,112,1,"{'RestaurantsAttire': 'u'dressy'', 'Restaurant...","Restaurants, American (New), Nightlife, Tapas ...","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."


Sólo hay 41 registros de Ciudades pertenecientes al Estado de California. Y dentro de ellos, ni siquiera se hace una depuración del rubro gastronómico.

In [28]:
df_ciudades['state'] = 'CA' #Cambiamos el Estado a California en este nuevo DataFrame
df_ciudades.head()

C:\Users\GASTON\AppData\Local\Temp\ipykernel_21604\1991998615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ciudades['state'] = 'CA' #Cambiamos el Estado a California en este nuevo DataFrame


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
762,mYbEGFsRcA0tfUNSYcBJMA,Golden River Restaurant,"8 E Scott St, Ste 10",Riverside,CA,08075,40.037703,-74.959281,5.0,8,1,"{'RestaurantsReservations': 'False', 'Restaura...","American (New), Restaurants",None
2072,xjF8NZZ2WIk3A2sW3nXyhA,Lambert International Airport Cell Phone Lot T...,9896 Air Cargo Rd,Berkeley,CA,63134,38.734285,-90.348893,4.0,13,1,"{'BusinessAcceptsCreditCards': 'False', 'WiFi'...","Automotive, Parking, Airports, Hotels & Travel",None
22467,dWix751x3N2wKv7Qwn1zDA,Genovesi Glass & Mirror,560 S Fairview St,Riverside,CA,08075,40.031,-74.9561,5.0,6,1,{'BusinessAcceptsCreditCards': 'True'},"Windows Installation, Home Services, Contracto...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
22793,Zk1lvwSdMk8HeXMAPQBUqw,Delta Movers,,Berkeley,CA,63130,38.750336,-90.342778,1.5,28,1,"{'BusinessAcceptsCreditCards': 'False', 'ByApp...","Self Storage, Home Services, Local Services, M...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
28393,6iHvWggONIDt8cKXJGz0VQ,The Madison,33 Lafayette St,Riverside,CA,08075,40.038566,-74.960761,3.5,112,1,"{'RestaurantsAttire': 'u'dressy'', 'Restaurant...","Restaurants, American (New), Nightlife, Tapas ...","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."


Extraemos el archivo, para así filtrar las reseñas.

In [29]:
df_ciudades.to_csv(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\data parcial\business_california.csv')

En principio, no tiene sentido continuar analizando el dataset de Yelp, ya que sólo tiene 41 registros pertenecientes (efectivamente) al Estado de California, en el que se requiere información para el proyecto. Sin embargo, se cruzarán los datos con las reviews, para analizar si hay un número considerable de reseñas sobre estos pocos comercios, y de esa forma valga la pena considerar estos datos en nuestro análisis.